# Code for Generating a Likelihood Table for a given pulsar

The code below walks through a full likelihood-table generation, diagnosis, and single-pulsar run using the likelihood table.

In [ ]:
from __future__ import division

import numpy as np
import os, glob, json
import matplotlib.pyplot as plt
import pickle
import scipy.linalg as sl
import healpy as hp
import multiprocessing as mp

from enterprise.signals import parameter
from enterprise.signals import signal_base
from enterprise.signals import deterministic_signals
from enterprise.signals import utils
from enterprise import constants as const
from enterprise.signals.signal_base import LogLikelihood
from enterprise.signals.signal_base import LookupLikelihood

from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import sampler as ee_sampler

from la_forge.core import Core, load_Core
from la_forge import rednoise
from la_forge.diagnostics import plot_chains

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

def make_lookup_table(psr, noisefile, outdir, sign, log10_amps, log10_amp_spacing, gammas, gamma_spacing, Ts, time_spacing):

    if not os.path.exists(outdir + psr.name):
        os.mkdir(outdir + psr.name)
        
    #now we need to make a pta for this pulsar to look up likelihoods for each amplitude we calculate
    #################
    ####   PTA   ####
    #################


    pta = ee_models.model_ramp([psr], LogLikelihood,
                          upper_limit=False, bayesephem=False,
                          Tmin_bwm=t0min, Tmax_bwm=t0max, logmin=min(log10_amps), logmax=max(log10_amps))
    
    print("Here are the parameters of the pta: {}".format(pta.params))




    with open(noisefile, 'rb') as nfile:
       setpars = json.load(nfile)

    pta.set_default_params(setpars)

    with open(outdir + "{}/{}_{}.txt".format(psr.name, psr.name, sign),'a+') as f:
        for t0 in Ts:
            for log10_strain in log10_amps:
                #set up the sky location of the pulsar
                #ramp_amp*=sign
                #print(psr.name + "would see an amplitude of: " + str(ramp_amp))

                #Now we need to add the A_red and gamma_red params so that we have in total:
                #A_red, Gamma_red, A_ramp, t_ramp
                for log10_rn_amp in log10_amps:
                    for gamma in gammas:
                        #now we have the four parameters, we need to ask the pta to calculate a likelihood
                        #the pta params are in the order:
                        #[sign, psr_gamma, psr_log10_A, ramp_log10_A, ramp_t0]
                        lnlike = pta.get_lnlikelihood([gamma, log10_rn_amp, log10_strain, t0, sign])
                        f.write('{:.12f}\n'.format(float(lnlike)).rjust(20))



Load the pulsar

In [ ]:
pkl_path = '/home/nima/nanograv/11yr_burst_factorizedlikelihood/NANOGrav_11yr_DE436.pickle'
lookup_outdir = '/home/nima/nanograv/11yr_burst_factorizedlikelihood/single_psr_lookup_v4/'
noisefile = '/home/nima/nanograv/11yr_burst_factorizedlikelihood/noisefiles/noisedict.json'

psrname = 'J1713+0747'

## Load the pulsar

with open(pkl_path, 'rb') as f:
    allpsrs=pickle.load(f)
    
for psr_i in allpsrs:
    if psr_i.name == psrname:
        psr = psr_i

## Build grid spacing

amp_spacing = '-20,-12,80'
log10_amps = np.linspace(-20, -12, 80, endpoint=True)

gamma_spacing ='0,7,70'
gammas = np.linspace(0, 7, 70, endpoint=True)

tmin = psr.toas.min() / const.day
tmax = psr.toas.max() / const.day


U,_ = utils.create_quantization_matrix(psr.toas)
eps = 9  # clip first and last N observing epochs
t0min = np.floor(max(U[:,eps] * psr.toas/const.day))
t0max = np.ceil(max(U[:,-eps] * psr.toas/const.day))

Ts = np.linspace(t0min, t0max, num=100, endpoint=True)
time_spacing = '{},{},100'.format(t0min, t0max)
sign_spacing = '-1,1,2'


## Some bookkeeping

if not os.path.exists(lookup_outdir + psr.name):
    os.mkdir(lookup_outdir + psr.name)


with open(lookup_outdir+'{}/pars.txt'.format(psr.name), 'w+') as f:
        f.write('{}_red_noise_gamma;{}\n{}_red_noise_log10_A;{}\n{};{}\n{};{}\n{};{}'.format(psr.name,gamma_spacing,psr.name,amp_spacing, 'ramp_log10_A',amp_spacing, 'ramp_t0',time_spacing,'sign', sign_spacing))

## Let it rip! We're doing the signs in parallel to speed things up, and we'll just add them back up
## at the end.

params1=[psr, noisefile, lookup_outdir, 1, log10_amps, amp_spacing, gammas, gamma_spacing, Ts, time_spacing]
params2=[psr, noisefile, lookup_outdir, -1, log10_amps, amp_spacing, gammas, gamma_spacing, Ts, time_spacing]

pool = mp.Pool(2)
pool.starmap(make_lookup_table, [params1, params2])


Combine the two different signed lookuptables to make the full table

In [ ]:
## just a quick concatenation of the two files
## currently, they're produced in parallel so there's one for negative-signed, one is positive-signed
neg_signs_file = lookup_outdir + '{}/{}_-1.txt'.format(psrname, psrname)
pos_signs_file = lookup_outdir + '{}/{}_1.txt'.format(psrname, psrname)

combined_table = lookup_outdir + '{}/{}_lookup.txt'.format(psrname, psrname)

filenames = [neg_signs_file, pos_signs_file]
with open(combined_table, 'w+') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

# Looking at likelihood surfaces

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
lfile = os.path.join(lookup_outdir,psrname,'{:}_lookup.txt'.format(psrname))
pfile = os.path.join(lookup_outdir,psrname,'pars.txt')

lookup = np.loadtxt(lfile)
params = np.loadtxt(pfile, dtype=str)

Ngrid = [70, 60, 60, 100, 2]  # gamma, A_RN, A_BWM, t0, signs (This is the order in the likelihood table)

In [ ]:
# Get the observing epochs for this psr
U,_ = utils.create_quantization_matrix(psr.toas)
eps = 9  # clip first and last N observing epochs
t0min = np.floor(max(U[:,eps] * psr.toas/const.day))
t0max = np.ceil(max(U[:,-eps] * psr.toas/const.day))

RN_gams = np.linspace(0, 7, 70, endpoint=True)
RN_amps = np.linspace(-20, -12, 80, endpoint=True)
BWM_amps = np.linspace(-18, -12, 80, endpoint=True)
BWM_t0s = np.linspace(t0min,t0max,100, endpoint=True)
BWM_signs = np.linspace(-1,1,2, endpoint=True)

In [ ]:
print(70*80*80*100*2, len(lookup)) #These ought to be the same!

In [ ]:
logL = lookup[:].reshape((2,100,80,80,70)) #reshape the loglikelihoods to get each column
#this corresponds to [sign, burst_t0, burst_amp, rn_amp, rn_gamma]
logL -= np.min(logL)

Get marginalized-ish likelihoods (the -ish is because we're simply summing over the other parameters to marginalize over them)

In [ ]:
logL_RNgam = np.sum(logL, axis=(0,1,2,3))
logL_RNamp = np.sum(logL, axis=(0,1,2,4))
logL_BWMamp = np.sum(logL, axis=(0,1,3,4))
logL_BWMt0 = np.sum(logL, axis=(0,2,3,4))
logL_BWMsign = np.sum(logL, axis=(1,2,3,4))

#Just make sure they're the right shapes still
len(logL_RNgam), len(logL_RNamp), len(logL_BWMamp), len(logL_BWMt0), len(logL_BWMsign)

Plot the marginalized posteriors to make sure they look reasonable. Note that the gamma posterior kind of looks goofy, not sure why, but if everything else looks ok, it's probably fine.

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(12,6))
ax[0,0].plot(RN_gams, logL_RNgam, label=r'$\gamma$')
ax[0,1].plot(RN_amps, logL_RNamp, label=r'$A_\mathrm{RN}$')
ax[1,0].plot(BWM_amps, logL_BWMamp, label=r'$A_\mathrm{BWM}$')
ax[1,1].plot(BWM_t0s, logL_BWMt0, label=r'$t_0$')
ax[2,0].plot(BWM_signs, logL_BWMsign, label='$sign$')
for a in ax.flatten():
    a.legend()

We can also look at the 2-d marginalized A vs gamma to see if we're getting a reasonable red-noise detection.

In [ ]:
# "marginalize" over burst params
logL_RN = np.sum(logL, axis=(0,1,2))
logL_RN.shape

plt.contourf(RN_gams, RN_amps, logL_RN, cmap='Blues')
plt.xlabel(r'$\gamma$')
plt.ylabel(r'$A_\mathrm{RN}$')

# Using the likelihood to do a single pulsar run

Load in the pulsar we want and instantiate our LookupLikelihood pta. Presumably, the psrname and locations of all the files are the same as above.

In [ ]:
lookupdir = lookup_outdir #Expect it to be in the place from above
# For the lookupdirectory passed in, it should be the master-directory that contains the directory
# for each pulsar. This is because in a multi-psr run, the pta needs to see every pulsar
sampling_outdir = '/home/nima/nanograv/11yr_burst_factorizedlikelihood/sgl_psr_run/ramp_factorized_v4/{}/'.format(psrname)


U,_ = utils.create_quantization_matrix(psr.toas)
eps = 9  # clip first and last N observing epochs
t0min = np.floor(max(U[:,eps] * psr.toas/const.day))
t0max = np.ceil(max(U[:,-eps] * psr.toas/const.day))
#Calculate the epochs for the pulsar

with open(noisefile, 'rb') as f:
    noisedict = json.load(f)

pta = ee_models.model_ramp([psr], LookupLikelihood, lookupdir=lookupdir, upper_limit=False, bayesephem=False,
                          Tmin_bwm=t0min, Tmax_bwm=t0max, noisedict=noisedict)



# Save the pta parameters and prior distributions
np.savetxt(sampling_outdir+'pars.txt',list(map(str, pta.param_names)), fmt='%s')
np.savetxt(sampling_outdir+'priors.txt',list(map(lambda x: str(x.__repr__()), pta.params)), fmt='%s')

Set up sampler and begin sampling

In [ ]:
sampling_outdir = '/home/nima/nanograv/11yr_burst_factorizedlikelihood/sgl_psr_run/ramp_factorized_v4/{}'.format(psrname)
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.1**2)
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,  outDir=sampling_outdir, resume=False)

jp = ee_sampler.JumpProposal(pta)
sampler.addProposalToCycle(jp.draw_from_prior, 30)

sampler.sample(x0, int(3e5), SCAMweight=30, AMweight=50, DEweight=0)

# Checking out the posteriors of the above runs

In [ ]:
from la_forge.core import Core
from la_forge.diagnostics import plot_chains

In [ ]:
chaindir=sampling_outdir
factorized_core = Core(label=psr.name + ' Factorized Run', chaindir=chaindir)

plot_chains(factorized_core, hist=False, 
            suptitle='Factorized Likelihood Posterior Traces',
            exclude=["lnlike", "chain_accept", "pt_chain_accept", "lnprior"])

In [ ]:
plot_chains(factorized_core, hist=True, 
            suptitle='Factorized Likelihood Posterior Traces',
            exclude=["lnlike", "chain_accept", "pt_chain_accept", "lnprior"])